In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64
training_dataloader = DataLoader(training_data, batch_size=batch_size)
testing_dataloader = DataLoader(testing_data, batch_size=batch_size)

In [4]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

class ClothingClassNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        return self.linear_relu_stack(x)

In [5]:
model = ClothingClassNeuralNetwork().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [6]:
def train_model(model, dataloader, loss_function, optimizer):
    dataset_size = len(dataloader.dataset)
    model.train()

    for index, (input_tensor, target) in enumerate(dataloader):
        input_tensor, target = input_tensor.to(device), target.to(device)

        prediction = model(input_tensor)
        loss = loss_function(prediction, target)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if index % 100 == 0:
            loss, current = loss.item(), (index + 1) * len(input_tensor)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")

In [7]:
def test_model(model, dataloader, loss_function):
    size = len(dataloader.dataset)
    batch_length = len(dataloader)
    loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for input_tensor, target in dataloader:
            input_tensor, target = input_tensor.to(device), target.to(device)
            prediction = model(input_tensor)
            loss += loss_function(prediction, target).item()
            correct += (prediction.argmax(1) == target).type(torch.float).sum().item()
    loss /= batch_length
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    

In [8]:
epochs = 20
for epoch_iteration in range(epochs):
    train_model(model, training_dataloader, loss_function, optimizer)
    test_model(model, testing_dataloader, loss_function)
print("Complete.")

loss: 2.308456  [   64/60000]
loss: 2.291605  [ 6464/60000]
loss: 2.267536  [12864/60000]
loss: 2.262124  [19264/60000]
loss: 2.240618  [25664/60000]
loss: 2.214715  [32064/60000]
loss: 2.218709  [38464/60000]
loss: 2.184670  [44864/60000]
loss: 2.180148  [51264/60000]
loss: 2.155977  [57664/60000]
Test Error: 
 Accuracy: 47.7%, Avg loss: 2.141757 

loss: 2.154233  [   64/60000]
loss: 2.148126  [ 6464/60000]
loss: 2.079239  [12864/60000]
loss: 2.097295  [19264/60000]
loss: 2.046851  [25664/60000]
loss: 1.988652  [32064/60000]
loss: 2.012365  [38464/60000]
loss: 1.933261  [44864/60000]
loss: 1.933109  [51264/60000]
loss: 1.874694  [57664/60000]
Test Error: 
 Accuracy: 59.1%, Avg loss: 1.858541 

loss: 1.893573  [   64/60000]
loss: 1.871430  [ 6464/60000]
loss: 1.736240  [12864/60000]
loss: 1.776802  [19264/60000]
loss: 1.671126  [25664/60000]
loss: 1.630197  [32064/60000]
loss: 1.643244  [38464/60000]
loss: 1.546574  [44864/60000]
loss: 1.567060  [51264/60000]
loss: 1.475967  [57664/600

In [9]:
torch.save(model.state_dict(), "clothing_class_model.pth")